In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

# --- Load preprocessed data ---
X = np.load("cnn_input.npy")  # shape: (n_samples, timesteps, 2)
y = np.load("cnn_target.npy")  # shape: (n_samples,)

print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (25197, 10000, 2), y shape: (25197,)


In [6]:
# --- Normalize voltage and stimulus independently ---
X[..., 0] = (X[..., 0] - np.mean(X[..., 0])) / np.std(X[..., 0])  # normalize stimulus
X[..., 1] = (X[..., 1] - np.mean(X[..., 1])) / np.std(X[..., 1])  # normalize voltage

# --- Split dataset ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# --- Build CNN model ---
model = Sequential([
    Input(shape=(X.shape[1], X.shape[2])),
    Conv1D(filters=32, kernel_size=5, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),

    Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1)  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])


In [11]:
# --- Train ---
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
630/630 ━━━━━━━━━━━━━━━━━━━━ 441s 696ms/step - loss: 136.7659 - mae: 3.1349 - val_loss: 51.4783 - val_mae: 2.0841
Epoch 2/100
630/630 ━━━━━━━━━━━━━━━━━━━━ 410s 650ms/step - loss: 50.3269 - mae: 2.0445 - val_loss: 50.3994 - val_mae: 2.2072
Epoch 3/100
630/630 ━━━━━━━━━━━━━━━━━━━━ 435s 691ms/step - loss: 52.4898 - mae: 2.2372 - val_loss: 49.6033 - val_mae: 2.3234
Epoch 4/100
630/630 ━━━━━━━━━━━━━━━━━━━━ 442s 702ms/step - loss: 54.8880 - mae: 2.3921 - val_loss: 49.0995 - val_mae: 2.4910
Epoch 5/100
328/630 ━━━━━━━━━━━━━━━━━━━━ 38:21 8s/step - loss: 45.2003 - mae: 2.4367

KeyboardInterrupt: 

In [ ]:
# --- Plot Training History ---
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.title("1D CNN Training Performance")
plt.legend()
plt.tight_layout()
plt.show()

# --- Evaluate ---
y_pred = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModel Evaluation:")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

# --- Plot: Predicted vs Actual ---
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--', label="Ideal")
plt.xlabel("Actual Spike Count")
plt.ylabel("Predicted Spike Count")
plt.title("1D CNN: Predicted vs Actual Spike Count")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()